In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import compression_entropy as ce

In [ ]:
files = ['entropies_10000.json', 'entropies_max1000_step100.json']

In [ ]:
path = '/home/pablo/ownCloud/WordOrderBibles/GitHub/output/KoplenigEtAl/WordPasting/'

In [ ]:
files = [path + file for file in files]

In [ ]:
jsons = []
for file in files:
    with open(file) as f:
        jsons.append(json.loads(f.read()))

In [ ]:
rows = []
for bible_entropies in jsons:
    row_list = []
    for filename, book_entropies in bible_entropies.items():
        for book_id, version_entropies in book_entropies.items():
            for n_iter, level_entropies in version_entropies.items():
                row = level_entropies
                row['filename'] = filename
                row['book_id'] = book_id
                row['iter_id'] = n_iter
                row_list.append(row)
    rows.append(row_list)

In [ ]:
dfs = [pd.DataFrame(row_list) for row_list in rows]

In [ ]:
df = pd.concat(dfs)

In [ ]:
def rel_error(a):
    assert len(a) == 2
    return abs(a[0] - a[1]) / (a[0] + a[1])

In [ ]:
for filename in df.filename.unique():
    for book_id in df.book_id.unique():
        for iter_id in df.iter_id.unique():
            selection = df[df.apply(lambda row: row['filename'] == filename and \
                                    row['book_id'] == book_id and \
                                    row['iter_id'] == iter_id, 
                                    1)]
            if len(selection) != 1:
                assert len(selection) == 2 and iter_id in ('0', '1000')
                for col in ('orig', 'shuffled', 'masked'):
                    assert rel_error(selection['orig'].tolist()) * 100 < 0.5

In [ ]:
df['bible_id'] = df['filename'].apply(lambda x: '.'.join(x.split('/')[-1].split('.')[:-1]))

In [ ]:
book_id_name = {'40': 'Matthew', 
                '41': 'Mark', 
                '42': 'Luke', 
                '43': 'John', 
                '44': 'Acts', 
                '66': 'Revelation'}
df['book'] = df['book_id'].map(book_id_name)

In [ ]:
df['D_structure'] = df.apply(lambda row: row['masked'] - row['orig'], 1)
df['D_order'] = df.apply(lambda row: row['shuffled'] - row['orig'], 1)

In [ ]:
for bible_id in df['bible_id'].unique():
    bible_df = df[df['bible_id'] == bible_id]
    for book_name in bible_df['book'].unique():
        book_df = bible_df[bible_df['book'] == book_name]
        x = book_df['D_order'].tolist()
        y = book_df['D_structure'].tolist()
        labels = book_df['iter_id'].tolist()
        fig, ax = plt.subplots()
        ax.scatter(x, y)
        plt.xlabel('Word order information')
        plt.ylabel('Word structure information')
        plt.title(f'{book_name}_{bible_id.split("-")[0]}')
        for i, txt in enumerate(labels):
            ax.annotate(txt, (x[i], y[i]), rotation=45)